In [82]:
from bs4 import BeautifulSoup
import requests

import numpy as np
import pandas as pd
import json

import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim

In [83]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(data, 'html.parser')

In [84]:
n_List = []
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    n_List.append(row.text)

In [85]:
df = pd.DataFrame({"Neighborhood": n_List})
df.head()

,Neighborhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


In [86]:
df.shape

(71, 1)

In [ ]:
import sys
!{sys.executable} -m pip install geocoder
import geocoder
from geopy.geocoders import arcgis as arcgis

In [88]:
def get_latlng(neighborhood):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [89]:
coords = [get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist() ]
coords[:5]

[[3.0576900000000364, 101.74388000000005],
 [3.1484988508598852, 101.69672774991264],
 [3.1903500000000236, 101.62545000000006],
 [3.1039100000000417, 101.71226000000007],
 [3.072750000000042, 101.71461000000005]]

In [90]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
df.head()

,Neighborhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.148499,101.696728
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610


In [91]:
address = 'Kuala Lumpur, Malaysia'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Kuala Lumpur, Malaysia are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Kuala Lumpur, Malaysia are 3.1516964, 101.6942371.


In [92]:
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11,tiles="Stamen Toner")

for lat, lng, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [93]:
# define Foursquare Credentials and Version and explore the neighborhoods
CLIENT_ID = 'PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H' # your Foursquare ID
CLIENT_SECRET = 'OASGQ1NIHPEOJZI3MOSTE4E0KDX0ZY1R0JIPHL3OP5LEXVUV' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PYANEEI0VEF3DT0UYBXNCSB2JR0U230EJF0NYRKSZMNBSL1H
CLIENT_SECRET:OASGQ1NIHPEOJZI3MOSTE4E0KDX0ZY1R0JIPHL3OP5LEXVUV


In [94]:
#Top 100 venues that are within a radius of 2000 meter
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
venues_df = pd.DataFrame(venues,columns=['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory'])
venues_df.head()

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
2,Alam Damai,3.05769,101.74388,Jc Deli 皆喜食坊,3.058397,101.748560,Food & Drink Shop
3,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
4,Alam Damai,3.05769,101.74388,Roti Canai Perindu,3.055071,101.740574,Breakfast Spot


In [95]:
venues_df.groupby(["Neighborhood"]).count() #Number of venues returned for each neighbor remember max limit was 100

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,100,100,100,100,100,100
...,...,...,...,...,...,...
Taman Tun Dr Ismail,100,100,100,100,100,100
Taman U-Thant,100,100,100,100,100,100
Taman Wahyu,100,100,100,100,100,100


We have fetched the data of 100 top venues for each neighbours(n = 71;df.shape[0]). Now look how many unique categories are there in total of all top venues for each neighbours. 

In [96]:
print('There are {} unique categories.'.format(len(venues_df['VenueCategory'].unique())))  

There are 310 unique categories.


In [97]:
venues_df['VenueCategory'].shape #It should be 7100(71 x 100) if all neighbours have 100 top venues data

(7092,)

In [98]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(7092, 311)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,...,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tunnel,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alam Damai,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [99]:
# group rows by neighborhood taking the mean of the frequency of occurrence of each category
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()
print(kl_grouped.shape)
kl_grouped

(71, 311)


,Neighborhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,...,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tunnel,Udon Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wedding Hall,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yakitori Restaurant,Yoga Studio,Zoo
0,Alam Damai,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.05,0.01,0.00,0.0,0.00,0.0,0.00,0.00,0.01,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.02,0.0,0.0,0.01,0.0,0.00,...,0.01,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.01,0.0,0.01,0.00,0.00,0.0,0.0,0.0,0.0,0.01,0.00,0.00,0.00,0.0,0.0
1,"Ampang, Kuala Lumpur",0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.0,0.00,0.0,0.00,0.01,0.00,0.00,0.00,0.03,0.00,0.00,0.00,0.01,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.01,0.0,0.0,0.00,0.0,0.00,...,0.00,0.00,0.01,0.0,0.0,0.01,0.01,0.00,0.0,0.01,0.0,0.01,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.01,0.00,0.01,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.01,0.0,0.0
2,Bandar Menjalara,0.00,0.0,0.01,0.0,0.00,0.00,0.00,0.00,0.03,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.03,0.00,0.02,0.00,0.00,0.0,0.01,0.0,0.01,0.0,0.01,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00,...,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.02,0.00,0.00,0.01,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.03,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0
3,Bandar Sri Permaisuri,0.01,0.0,0.00,0.0,0.00,0.01,0.00,0.00,0.05,0.00,0.01,0.0,0.02,0.0,0.01,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.01,0.00,0.0,0.01,0.0,0.00,0.0,0.00,0.00,0.0,0.0,0.02,0.0,0.0,0.00,0.0,0.00,...,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.01,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.01,0.00,0.0,0.00,0.00,0.01,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0
4,Bandar Tasik Selatan,0.01,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.09,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.01,0.00,0.0,0.0,0.03,0.0,0.0,0.01,0.0,0.00,...,0.00,0.01,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.04,0.00,0.00,0.00,0.01,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,Taman Tun Dr Ismail,0.01,0.0,0.00,0.0,0.00,0.00,0.00,0.02,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.03,0.01,0.01,0.00,0.00,0.00,0.0,0.01,0.0,0.00,0.0,0.01,0.00,0.0,0.0,0.00,0.0,0.0,0.01,0.0,0.01,...,0.00,0.00,0.01,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,0.01,0.00,0.01,0.00,0.00,0.0,0.01,0.00,0.00,0.01,0.00,0.0,0.00,0.00,0.00,0.0,0.01,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0
67,Taman U-Thant,0.01,0.0,0.00,0.0,0.00,0.01,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.00,0.01,0.03,0.00,0.00,0.00,0.00,0.0,0.01,0.0,0.01,0.0,0.03,0.00,0.0,0.0,0.01,0.0,0.0,0.00,0.0,

**Create a new DataFrame for Shopping Mall data only**

In [100]:
df_mall = kl_grouped[["Neighborhoods","Shopping Mall"]]
print(df_mall.shape)
df_mall.head()

(71, 2)


,Neighborhoods,Shopping Mall
0,Alam Damai,0.00
1,"Ampang, Kuala Lumpur",0.01
2,Bandar Menjalara,0.01
3,Bandar Sri Permaisuri,0.00
4,Bandar Tasik Selatan,0.01


Cluster Neighborhoods


In [101]:
# set number of clusters
kclusters = 3

kl_clustering = df_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 2, 0, 2, 0, 1, 1, 2, 0], dtype=int32)

In [102]:
kl_merged = df_mall.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [103]:
kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Alam Damai,0.00,0
1,"Ampang, Kuala Lumpur",0.01,2
2,Bandar Menjalara,0.01,2
3,Bandar Sri Permaisuri,0.00,0
4,Bandar Tasik Selatan,0.01,2


In [104]:
kl_merged = kl_merged.join(df.set_index("Neighborhood"), on="Neighborhood")

print(kl_merged.shape)
kl_merged.head() 

(71, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,0,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.01,2,3.148499,101.696728
2,Bandar Menjalara,0.01,2,3.190350,101.625450
3,Bandar Sri Permaisuri,0.00,0,3.103910,101.712260
4,Bandar Tasik Selatan,0.01,2,3.072750,101.714610


In [105]:
# sort the results by Cluster Labels
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.00,0,3.057690,101.743880
33,Kampung Padang Balang,0.00,0,3.209430,101.693180
34,Kepong,0.00,0,3.217500,101.637630
36,Kuchai Lama,0.00,0,3.090740,101.677330
38,Maluri,0.00,0,3.147890,101.694050
...,...,...,...,...,...
44,Putrajaya,0.02,2,3.125851,101.718513
15,Bukit Nanas,0.02,2,3.148609,101.699854
46,Segambut,0.02,2,3.186390,101.668100
19,Chow Kit,0.02,2,3.163590,101.698110


In [106]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11,tiles='stamen toner')

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters.save('cluster.html')
map_clusters

Examine the Clusters

#### Cluster 0

In [107]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.0,0,3.057690,101.743880
33,Kampung Padang Balang,0.0,0,3.209430,101.693180
34,Kepong,0.0,0,3.217500,101.637630
36,Kuchai Lama,0.0,0,3.090740,101.677330
38,Maluri,0.0,0,3.147890,101.694050
40,Miharja,0.0,0,3.147890,101.694050
42,Pantai Dalam,0.0,0,3.094760,101.667470
45,Salak South,0.0,0,3.081540,101.696890
47,Semarak,0.0,0,3.179943,101.721449
49,Setapak,0.0,0,3.188160,101.704150


#### Cluster 1

In [108]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
11,Brickfields,0.03,1,3.129160,101.684060
43,"Pudu, Kuala Lumpur",0.04,1,3.133540,101.713070
7,Bangsar Park,0.05,1,3.129200,101.678440
37,Lembah Pantai,0.04,1,3.121189,101.663889
31,"Kampung Baru, Kuala Lumpur",0.03,1,3.165460,101.710280
66,Taman Tun Dr Ismail,0.03,1,3.152830,101.622710
67,Taman U-Thant,0.03,1,3.157700,101.724520
27,"Jalan Cochrane, Kuala Lumpur",0.04,1,3.134630,101.721690
6,Bangsar,0.05,1,3.129200,101.678440


#### Cluster 2

In [109]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
4,Bandar Tasik Selatan,0.010000,2,3.072750,101.714610
64,Taman Sri Sinar,0.010101,2,3.190070,101.652930
8,Bangsar South,0.020000,2,3.111020,101.662830
57,Taman Desa,0.010000,2,3.102970,101.684710
2,Bandar Menjalara,0.010000,2,3.190350,101.625450
54,Sungai Besi,0.010000,2,3.049970,101.706030
1,"Ampang, Kuala Lumpur",0.010000,2,3.148499,101.696728
52,Sri Hartamas,0.020000,2,3.162200,101.650360
51,Shamelin,0.020000,2,3.124570,101.735970
25,"Federal Hill, Kuala Lumpur",0.010000,2,3.136370,101.685640


###Quick Result


*   Mostly shopping malls are situated in cluster 1 ranging from 3 to 5 and hence highest competition.
*   Cluster 0 has least competition with no present situated.
*   Cluster 2 has medium level of competition having malls ranging from 1 to 2.

